<a href="https://colab.research.google.com/github/abosedealli/SIT723UNIT_STOCK_-PREDICTION_AR_ARIMA_LSTM_LR-/blob/main/2years%20nexflix%20%20LSTM%2060days%20forcast%20-short-term.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# load your dataset
data= pd.read_csv("/content/neflix.csv")
data.shape
netflix= data.iloc[:int(0.8*len(data))]
print(netflix.shape)
netflix.info()


netflix.head()
# create a dataframe with only the open, low, and high prices
data = data[['Open','Adj Close', 'High']]
prices = data

# create a variable to store the number of days you want to forecast
forecast_days = 60

# create a variable to store the percentage of data you want to use for testing
test_data_size = 0.2

# create a variable to store the number of days in your dataset
days_in_dataset = len(data)

# create variables to store the number of days you want to use for testing and training
test_days = int(days_in_dataset * test_data_size)
train_days = days_in_dataset - test_days

# create a training dataset
train_data = data.iloc[:train_days]

# create a testing dataset
test_data = data.iloc[train_days:]

# create a scaler to normalize the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# fit the scaler to the training data
scaler.fit(train_data)

# normalize the training data
train_data = scaler.transform(train_data)

# create a dataset with only the open, low, and high prices for the last forecast_days days
X_train = []
y_train = []
for i in range(forecast_days, train_days):
    X_train.append(train_data[i-forecast_days:i])
    y_train.append(train_data[i])
X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 3))


# create a LSTM model
model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], 3)))
model.add(LSTM(50))
model.add(Dense(3))

# compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# fit the model to the training data
model.fit(X_train, y_train, epochs=5, batch_size=32)

# normalize the testing data
test_data = scaler.transform(test_data)

# create a dataset with only the open, low, and high prices for the last forecast_days days
X_test = []
y_test = []
for i in range(forecast_days, test_days):
    X_test.append(test_data[i-forecast_days:i])
    y_test.append(test_data[i])
X_test, y_test = np.array(X_test), np.array(y_test)

# reshape the data to be 3D
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 3))

# make predictions
predictions = model.predict(X_test)

# inverse transform the predictions to get the original prices
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test)

# calculate the mean squared error of the predictions
from sklearn.metrics import mean_squared_error
mse = (mean_squared_error(y_test, predictions, squared = False))
     
print("Mean Squared Error:", mse)
ape = np.abs((y_test - predictions) / y_test)
# Calculate the mean of the absolute percentage difference
mape = np.mean(ape) * 100
print("MAPE: ", mape)

# Calculate the mean absolute error difference
from sklearn.metrics import mean_absolute_error
mae= np.mean(np.abs(y_test - predictions))
print("MAE: ", mae)

rmse=mse = (mean_squared_error(y_test, predictions, squared = True))
print("RMSE:", rmse)

import plotly.graph_objs as go

# generate new data
X_forecast = []
for i in range(forecast_days, forecast_days+60):
    X_forecast.append(prices[i-forecast_days:i])
X_forecast = np.array(X_forecast)


forecast = model.predict(X_forecast)

# Get the last predicted value
last_prediction = predictions[-1]

# Add a dimension to the last value of predictions
last_prediction = np.expand_dims(last_prediction, axis=0)
forecast = scaler.inverse_transform(forecast)
# Insert the last value of predictions at the 0th index of forecast
forecast = np.insert(forecast, 0, last_prediction, axis=0)
trace1 = go.Scatter(x=np.arange(y_test.shape[0]), y=y_test[:,0], name='Actual Open Prices')
trace2 = go.Scatter(x=np.arange(predictions.shape[0]), y=predictions[:,0], name='Predicted Open Prices')
trace3 = go.Scatter(x=np.arange(predictions.shape[0]-1, predictions.shape[0] + forecast.shape[0]-1), y=forecast[:,0], name='Forecasted Open Prices')
trace4 = go.Scatter(x=np.arange(y_test.shape[0]), y=y_test[:,1], name='Actual Adj Close Prices')
trace5 = go.Scatter(x=np.arange(predictions.shape[0]), y=predictions[:,1], name='Predicted Adj Close Prices')
trace6 = go.Scatter(x=np.arange(predictions.shape[0]-1, predictions.shape[0] + forecast.shape[0]-1), y=forecast[:,1], name='Forecasted Adj Close Prices')
trace7 = go.Scatter(x=np.arange(y_test.shape[0]), y=y_test[:,2], name='Actual High Prices')
trace8 = go.Scatter(x=np.arange(predictions.shape[0]), y=predictions[:,2], name='Predicted High Prices')
trace9 = go.Scatter(x=np.arange(predictions.shape[0]-1, predictions.shape[0] + forecast.shape[0]-1), y=forecast[:,2], name='Forecasted High Prices')



data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9]
layout = go.Layout(title='Actual, Predicted and Forecasted Prices', xaxis=dict(title='Time'), yaxis=dict(title='Price'))
fig = go.Figure(data=data, layout=layout)
fig.show()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
(604, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 604 entries, 0 to 603
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       604 non-null    object 
 1   Open       604 non-null    float64
 2   High       604 non-null    float64
 3   Low        604 non-null    float64
 4   Close      604 non-null    float64
 5   Adj Close  604 non-null    float64
 6   Volume     604 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 33.2+ KB
Epoch 1/5
18/18 [==============================] - 6s 85ms/step - loss: 0.0943
Epoch 2/5
18/18 [==============================] - 1s 79ms/step - loss: 0.0106
Epoch 3/5
18/18 [==============================] - 1s 77ms/step - loss: 0.0052
Epoch 4/5
18/18 [==============================] - 1s 77ms/step - loss: 0.0039
Epoch 5/5
3/3 [==============================] - 